**LeetCode**

In [180]:
A =[[1,5], [10,14], [16,18]]
B =[[2,6], [8,10], [11,20]]

In [181]:
def merge(A,B):
  i = 0
  j = 0
  out = []
  start = -float('inf')
  p_end = -float('inf')
  while i<len(A) and j<len(B):
    if A[i][0] < B[j][0]:
      cand_start = A[i][0]
      temp_end = A[i][1]
      i += 1
    else:
      cand_start = B[j][0]
      temp_end = B[j][1]
      j += 1

    if cand_start > p_end:
      out.append([start,p_end])
      start = cand_start
      p_end = temp_end
    else:
      p_end = max(p_end, temp_end)

  out.append([start, p_end])
  print(out[1:])


In [182]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

print(os.getcwd())

df_v1 = pd.read_csv('hdrs.csv')

# Creating a set for just the first visits and appending the sum of all the patients visits
v1_set = [col for col in df_v1 if col.startswith('V1')]
df_89 = df_v1.loc[:, v1_set]
df = df_89.loc[:, [col for col in df_89 if col.startswith('V1-HAMD08') | col.startswith('V1-HAMD09')]]
df['V1-all'] = df_89.sum(axis=1)

df

C:\Users\hnr59\Documents\Mission2_07_17


,V1-HAMD08,V1-HAMD09,V1-all
0,0,0,7
1,1,0,14
2,1,1,22
3,0,0,6
4,0,0,8
...,...,...,...
1463,1,1,27
1464,1,1,26
1465,0,1,13
1466,2,1,20


In [183]:

# create a list of our conditions
conditions = [
    (df['V1-all'] <= 17),
    (df['V1-all'] > 17) & (df['V1-all'] <= 24),
    (df['V1-all'] > 24)
    ]

#for i in range(0,len(df)):
  #if (df['V1-HAMD08'][i] ==2 & df['V1-HAMD09'][i] <=3):
    #df.drop(i)
    #print(i)

# create a list of the values we want to assign for each condition
values = ['mild', 'moderate', 'severe']

# create a new column and use np.select to assign values to it using our lists as arguments
df['level'] = np.select(conditions, values)
df_23 = df.loc[((df['V1-HAMD08'] ==2)& (df['V1-HAMD09'] ==3))]
print(df_23)
df_24 = df.loc[((df['V1-HAMD08'] ==2)& (df['V1-HAMD09'] ==4))]
print(df_24)
df_33 = df.loc[((df['V1-HAMD08'] ==3)& (df['V1-HAMD09'] ==3))]
print(df_33)
df = df.drop(df_23.index)
df = df.drop(df_24.index)
df = df.drop(df_33.index)


      V1-HAMD08  V1-HAMD09  V1-all     level
73            2          3      22  moderate
408           2          3      24  moderate
492           2          3      29    severe
960           2          3      22  moderate
1006          2          3      18  moderate
1181          2          3      22  moderate
1215          2          3      24  moderate
     V1-HAMD08  V1-HAMD09  V1-all   level
327          2          4      30  severe
392          2          4      33  severe
393          2          4      32  severe
     V1-HAMD08  V1-HAMD09  V1-all   level
935          3          3      26  severe


## Chi-square

In [184]:
from scipy.stats import chi2_contingency

def calculate_chi2_contingency(df):
  cross_table = pd.crosstab(index=df['V1-HAMD08'], columns=df['V1-HAMD09'])
  print(cross_table)
  res = chi2_contingency(cross_table)
  print(f"p value:  {res.pvalue}")

In [185]:
calculate_chi2_contingency(df)

V1-HAMD09    0    1    2   3  4
V1-HAMD08                      
0          194  194   83  11  1
1          270  305  134  10  5
2           96   86   52   0  0
3            8    4    4   0  0
p value:  0.2388481596112691


In [186]:
calculate_chi2_contingency(df[df['level']=='mild'])

V1-HAMD09    0    1   2  3  4
V1-HAMD08                    
0          148  100  21  1  1
1          129  119  21  1  0
2           15   11   2  0  0
p value:  0.8378724813380201


In [187]:
calculate_chi2_contingency(df[df['level']=='moderate'])

V1-HAMD09    0    1   2   3
V1-HAMD08                  
0           43   82  49  10
1          127  160  89   6
2           58   58  37   0
3            7    3   3   0
p value:  0.004675672275018173


In [188]:
calculate_chi2_contingency(df[df['level']=='severe'])

V1-HAMD09   0   1   2  3  4
V1-HAMD08                  
0           3  12  13  0  0
1          14  26  24  3  5
2          23  17  13  0  0
3           1   1   1  0  0
p value:  0.032624819945455824


## Persion correlation

In [189]:
df[['V1-HAMD08','V1-HAMD09']].corr()

,V1-HAMD08,V1-HAMD09
V1-HAMD08,1.000000,-0.001191
V1-HAMD09,-0.001191,1.000000


In [190]:
df[df['level']=='mild'][['V1-HAMD08','V1-HAMD09']].corr()

,V1-HAMD08,V1-HAMD09
V1-HAMD08,1.000000,0.027458
V1-HAMD09,0.027458,1.000000


In [191]:
df[df['level']=='moderate'][['V1-HAMD08','V1-HAMD09']].corr()

,V1-HAMD08,V1-HAMD09
V1-HAMD08,1.000000,-0.128199
V1-HAMD09,-0.128199,1.000000


In [192]:
df[df['level']=='severe'][['V1-HAMD08','V1-HAMD09']].corr()

,V1-HAMD08,V1-HAMD09
V1-HAMD08,1.000000,-0.234887
V1-HAMD09,-0.234887,1.000000


## Spearman's rank correlation coefficient
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html

https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

In [193]:
from scipy import stats

res = stats.spearmanr(df[['V1-HAMD08','V1-HAMD09']])
res

SignificanceResult(statistic=0.007199497812029542, pvalue=0.7836409704972805)

In [194]:
res = stats.spearmanr(df[df['level']=='mild'][['V1-HAMD08','V1-HAMD09']])
res

SignificanceResult(statistic=0.04709250419607791, pvalue=0.2620821320567572)

In [195]:
res = stats.spearmanr(df[df['level']=='moderate'][['V1-HAMD08','V1-HAMD09']])
res

SignificanceResult(statistic=-0.11968334036340954, pvalue=0.0011777007261296673)

In [196]:
res = stats.spearmanr(df[df['level']=='severe'][['V1-HAMD08','V1-HAMD09']])
res

SignificanceResult(statistic=-0.2625287679154087, pvalue=0.0009300237063123249)

# Mission 2

In [197]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [198]:
hdrs = pd.read_csv('hdrs_RDR.csv')

df2 = df
df2['DRUG'] = df_v1.loc[:, [col for col in df_v1 if col.startswith('DRUG')]]
df2['Remission'] = hdrs.loc[:, [col for col in hdrs if col.startswith('Remission')]]
df2['Difference'] = hdrs.loc[:, [col for col in hdrs if col.startswith('Difference')]]
df2['Response'] = hdrs.loc[:, [col for col in hdrs if col.startswith('Response')]]

remission_labels = ["Full Remission", "Remission", "Partial Remission", "No/Mild Remission"]
difference_labels = ["Worst", "No Change", "Improvement", "Big Improvement"]
response_labels = ["No Response", "Partial Response", "Full Response"]

df2

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
0,0,0,7,mild,Duloxetine,Full Remission,Improvement,Full Response
1,1,0,14,mild,Duloxetine,Partial Remission,Worst,No Response
2,1,1,22,moderate,Duloxetine,Full Remission,Improvement,Full Response
3,0,0,6,mild,Duloxetine,Remission,Worst,No Response
4,0,0,8,mild,Duloxetine,Remission,Worst,No Response
...,...,...,...,...,...,...,...,...
1463,1,1,27,severe,Fluoxetine,Remission,Big Improvement,Full Response
1464,1,1,26,severe,Fluoxetine,Full Remission,Big Improvement,Full Response
1465,0,1,13,mild,Fluoxetine,Remission,Worst,No Response
1466,2,1,20,moderate,Fluoxetine,Remission,Improvement,Partial Response


# Agitation Only for each drug

In [199]:
a_o = df2.loc[((df['V1-HAMD08'] == 0) & (df['V1-HAMD09'] > 0))]
a_od = a_o[a_o['DRUG'] == 'Duloxetine']

aod1 = go.Histogram(x = a_od['Remission'], name = 'Remission')
aod2 = go.Histogram(x = a_od['Difference'], name = 'Difference')
aod3 = go.Histogram(x = a_od['Response'], name = 'Response')

fig1 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig1.add_trace(aod1, row = 1, col = 1)
fig1.add_trace(aod2, row = 1, col = 2)
fig1.add_trace(aod3, row = 1, col = 3)

fig1.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig1.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig1.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig1.update_layout(
    font = {
        "size": 17
    },
    
    title = {
        "text": "Figure 1. Agitation Only with Patients taking Duloxetine",
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig1.show()

#len(a_od.loc[a_od['Remission'] == 'Full Remission']
# len(a_od[a_od['Remission'] == 'Partial Remission']),
#                         len(a_od[a_od['Remission'] == 'No/Mild Remission'])

# aod1 = go.Histogram(x = [['Remission','Remission','Remission','Remission','Remission'],
#            ['Full Remission', 'No/Mild Remission', 'Partial Remission','Remission']],
#                     y= [8, 9, 6, 9], name = 'Remission')

In [200]:
# Duloxetine Remission

mild_ad = len(a_od[a_od['Remission'] == 'No/Mild Remission'])
part_ad = len(a_od[a_od['Remission'] == 'Partial Remission'])
remi_ad = len(a_od[a_od['Remission'] == 'Remission'])
full_ad = len(a_od[a_od['Remission'] == 'Full Remission'])

# Duloxetine Difference
wor_ad = len(a_od[a_od['Difference'] == 'Worst'])
nc_ad = len(a_od[a_od['Difference'] == 'No Change'])
i_ad = len(a_od[a_od['Difference'] == 'Improvement'])
bi_ad = len(a_od[a_od['Difference'] == 'Big Improvement'])

# Duloxetine Response
nr_ad = len(a_od[a_od['Response'] == 'No Response'])
pr_ad = len(a_od[a_od['Response'] == 'Partial Response'])
fr_ad = len(a_od[a_od['Response'] == 'Full Response'])

ad = [mild_ad, part_ad, remi_ad, full_ad,
        wor_ad, nc_ad, i_ad, bi_ad,
        nr_ad, pr_ad, fr_ad]

ad

[5, 11, 86, 89, 16, 4, 138, 33, 20, 63, 108]

In [201]:
a_ov = a_o[a_o['DRUG'] == 'Venlafaxine']
aov1 = go.Histogram(x = a_ov['Remission'], name = 'Remission')
aov2 = go.Histogram(x = a_ov['Difference'], name = 'Difference')
aov3 = go.Histogram(x = a_ov['Response'], name = 'Response')

fig2 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig2.add_trace(aov1, row = 1, col = 1)
fig2.add_trace(aov2, row = 1, col = 2)
fig2.add_trace(aov3, row = 1, col = 3)

fig2.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig2.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig2.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig2.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 2. Agitation Only with Patients taking Venlafaxine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)


fig2.show()

In [202]:
# Venlafaxine Remission

mild_av = len(a_ov[a_ov['Remission'] == 'No/Mild Remission'])
part_av = len(a_ov[a_ov['Remission'] == 'Partial Remission'])
remi_av = len(a_ov[a_ov['Remission'] == 'Remission'])
full_av = len(a_ov[a_ov['Remission'] == 'Full Remission'])

#Venlafaxine Difference
wor_av = len(a_ov[a_ov['Difference'] == 'Worst'])
nc_av = len(a_ov[a_ov['Difference'] == 'No Change'])
i_av = len(a_ov[a_ov['Difference'] == 'Improvement'])
bi_av = len(a_ov[a_ov['Difference'] == 'Big Improvement'])

# Venlafaxine Response
nr_av = len(a_ov[a_ov['Response'] == 'No Response'])
pr_av = len(a_ov[a_ov['Response'] == 'Partial Response'])
fr_av = len(a_ov[a_ov['Response'] == 'Full Response'])

av = [mild_av, part_av, remi_av, full_av,
        wor_av, nc_av, i_av, bi_av,
        nr_av, pr_av, fr_av]

av

[0, 4, 18, 1, 0, 1, 21, 1, 1, 14, 8]

In [203]:
a_op = a_o[a_o['DRUG'] == 'Paroxetine']
aop1 = go.Histogram(x = a_op['Remission'], name = 'Remission')
aop2 = go.Histogram(x = a_op['Difference'], name = 'Difference')
aop3 = go.Histogram(x = a_op['Response'], name = 'Response')

fig3 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig3.add_trace(aop1, row = 1, col = 1)
fig3.add_trace(aop2, row = 1, col = 2)
fig3.add_trace(aop3, row = 1, col = 3)

fig3.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig3.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig3.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig3.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 3. Agitation Only with Patients taking Paroxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig3.show()

In [204]:
# Paroxetine Remission
mild_ap = len(a_op[a_op['Remission'] == 'No/Mild Remission'])
part_ap = len(a_op[a_op['Remission'] == 'Partial Remission'])
remi_ap = len(a_op[a_op['Remission'] == 'Remission'])
full_ap = len(a_op[a_op['Remission'] == 'Full Remission'])

#Paroxetine Difference
wor_ap = len(a_op[a_op['Difference'] == 'Worst'])
nc_ap = len(a_op[a_op['Difference'] == 'No Change'])
i_ap = len(a_op[a_op['Difference'] == 'Improvement'])
bi_ap = len(a_op[a_op['Difference'] == 'Big Improvement'])

#Paroxetine Response
nr_ap = len(a_op[a_op['Response'] == 'No Response'])
pr_ap = len(a_op[a_op['Response'] == 'Partial Response'])
fr_ap = len(a_op[a_op['Response'] == 'Full Response'])

ap = [mild_ap, part_ap, remi_ap, full_ap,
        wor_ap, nc_ap, i_ap, bi_ap,
        nr_ap, pr_ap, fr_ap]

ap

[1, 2, 25, 33, 5, 1, 49, 6, 6, 25, 30]

In [205]:
a_of = a_o[a_o['DRUG'] == 'Fluoxetine']
aof1 = go.Histogram(x = a_of['Remission'], name = 'Remission')
aof2 = go.Histogram(x = a_of['Difference'], name = 'Difference')
aof3 = go.Histogram(x = a_of['Response'], name = 'Response')

fig4 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig4.add_trace(aof1, row = 1, col = 1)
fig4.add_trace(aof2, row = 1, col = 2)
fig4.add_trace(aof3, row = 1, col = 3)

fig4.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig4.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig4.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig4.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 4. Agitation Only with Patients taking Fluoxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig4.show()

In [206]:
# Fluoxetine Remission

mild_af = len(a_of[a_of['Remission'] == 'No/Mild Remission'])
part_af = len(a_of[a_of['Remission'] == 'Partial Remission'])
remi_af = len(a_of[a_of['Remission'] == 'Remission'])
full_af = len(a_of[a_of['Remission'] == 'Full Remission'])
                             
# Fluoxetine Difference
wor_af = len(a_of[a_of['Difference'] == 'Worst'])
nc_af = len(a_of[a_of['Difference'] == 'No Change'])
i_af = len(a_of[a_of['Difference'] == 'Improvement'])
bi_af = len(a_of[a_of['Difference'] == 'Big Improvement'])
                             
#Fluoxetine Response
nr_af = len(a_of[a_of['Response'] == 'No Response'])
pr_af = len(a_of[a_of['Response'] == 'Partial Response'])
fr_af = len(a_of[a_of['Response'] == 'Full Response'])
                             
af = [mild_af, part_af, remi_af, full_af,
        wor_af, nc_af, i_af, bi_af,
        nr_af, pr_af, fr_af]

af

[0, 3, 7, 4, 5, 0, 9, 0, 5, 5, 4]

# Retardation Only for each drug

In [207]:
r_o = df2.loc[((df['V1-HAMD08'] > 0) & (df['V1-HAMD09'] == 0))]
r_od = r_o[r_o['DRUG'] == 'Duloxetine']

rod1 = go.Histogram(x = r_od['Remission'], name = 'Remission')
rod2 = go.Histogram(x = r_od['Difference'], name = 'Difference')
rod3 = go.Histogram(x = r_od['Response'], name = 'Response')

fig5 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig5.add_trace(rod1, row = 1, col = 1)
fig5.add_trace(rod2, row = 1, col = 2)
fig5.add_trace(rod3, row = 1, col = 3)

fig5.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig5.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig5.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig5.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 5. Retardation Only with Patients taking Duloxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig5.show()

r_o

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
1,1,0,14,mild,Duloxetine,Partial Remission,Worst,No Response
18,2,0,24,moderate,Duloxetine,Remission,Improvement,Full Response
22,1,0,13,mild,Duloxetine,Full Remission,Improvement,Full Response
25,1,0,20,moderate,Duloxetine,Remission,Improvement,Partial Response
27,1,0,12,mild,Duloxetine,Remission,No Change,No Response
...,...,...,...,...,...,...,...,...
1446,1,0,23,moderate,Fluoxetine,Partial Remission,No Change,No Response
1452,2,0,20,moderate,Fluoxetine,Remission,Improvement,Partial Response
1458,1,0,19,moderate,Fluoxetine,Remission,Improvement,Full Response
1461,1,0,21,moderate,Fluoxetine,Full Remission,Big Improvement,Full Response


In [208]:
# Duloxetine Remission

mild_rd = len(r_od[r_od['Remission'] == 'No/Mild Remission'])
part_rd = len(r_od[r_od['Remission'] == 'Partial Remission'])
remi_rd = len(r_od[r_od['Remission'] == 'Remission'])
full_rd = len(r_od[r_od['Remission'] == 'Full Remission'])

#Duloxetine Difference
wor_rd = len(r_od[r_od['Difference'] == 'Worst'])
nc_rd = len(r_od[r_od['Difference'] == 'No Change'])
i_rd = len(r_od[r_od['Difference'] == 'Improvement'])
bi_rd = len(r_od[r_od['Difference'] == 'Big Improvement'])

# Duloxetine Response
nr_rd = len(r_od[r_od['Response'] == 'No Response'])
pr_rd = len(r_od[r_od['Response'] == 'Partial Response'])
fr_rd = len(r_od[r_od['Response'] == 'Full Response'])

rd = [mild_rd, part_rd, remi_rd, full_rd,
        wor_rd, nc_rd, i_rd, bi_rd,
        nr_rd, pr_rd, fr_rd]

rd

[4, 14, 113, 111, 12, 3, 185, 42, 15, 95, 132]

In [209]:
r_ov = r_o[r_o['DRUG'] == 'Venlafaxine']

rov1 = go.Histogram(x = r_ov['Remission'], name = 'Remission')
rov2 = go.Histogram(x = r_ov['Difference'], name = 'Difference')
rov3 = go.Histogram(x = r_ov['Response'], name = 'Response')

fig6 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig6.add_trace(rov1, row = 1, col = 1)
fig6.add_trace(rov2, row = 1, col = 2)
fig6.add_trace(rov3, row = 1, col = 3)

fig6.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig6.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig6.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig6.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 6. Retardation Only with Patients taking Venlafaxine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig6.show()

r_ov

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
165,1,0,20,moderate,Venlafaxine,Full Remission,Improvement,Full Response
171,1,0,27,severe,Venlafaxine,Partial Remission,Improvement,Partial Response
184,1,0,22,moderate,Venlafaxine,Remission,Improvement,Partial Response
186,2,0,26,severe,Venlafaxine,Remission,Improvement,Partial Response
199,1,0,19,moderate,Venlafaxine,Remission,Improvement,Partial Response
201,1,0,20,moderate,Venlafaxine,Full Remission,Improvement,Full Response
204,1,0,25,severe,Venlafaxine,Remission,Improvement,Partial Response
206,2,0,26,severe,Venlafaxine,Full Remission,Big Improvement,Full Response
208,1,0,18,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
224,1,0,20,moderate,Venlafaxine,Partial Remission,Improvement,Partial Response


In [210]:
# Venlafaxine Remission
mild_rv = len(r_ov[r_ov['Remission'] == 'No/Mild Remission'])
part_rv = len(r_ov[r_ov['Remission'] == 'Partial Remission'])
remi_rv = len(r_ov[r_ov['Remission'] == 'Remission'])
full_rv = len(r_ov[r_ov['Remission'] == 'Full Remission'])

# Venlafaxine Difference
wor_rv = len(r_ov[r_ov['Difference'] == 'Worst'])
nc_rv = len(r_ov[r_ov['Difference'] == 'No Change'])
i_rv = len(r_ov[r_ov['Difference'] == 'Improvement'])
bi_rv = len(r_ov[r_ov['Difference'] == 'Big Improvement'])

# Venlafaxine Response
nr_rv = len(r_ov[r_ov['Response'] == 'No Response'])
pr_rv = len(r_ov[r_ov['Response'] == 'Partial Response'])
fr_rv = len(r_ov[r_ov['Response'] == 'Full Response'])

rv = [mild_rv, part_rv, remi_rv, full_rv,
        wor_rv, nc_rv, i_rv, bi_rv,
        nr_rv, pr_rv, fr_rv]

rv

[0, 4, 15, 15, 0, 0, 24, 10, 0, 17, 17]

In [211]:
r_op = r_o[r_o['DRUG'] == 'Paroxetine']
rop1 = go.Histogram(x = r_op['Remission'], name = 'Remission')
rop2 = go.Histogram(x = r_op['Difference'], name = 'Difference')
rop3 = go.Histogram(x = r_op['Response'], name = 'Response')

fig7 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig7.add_trace(rop1, row = 1, col = 1)
fig7.add_trace(rop2, row = 1, col = 2)
fig7.add_trace(rop3, row = 1, col = 3)

fig7.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig7.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig7.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig7.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 7. Retardation Only with Patients taking Paroxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig7.show()


r_op

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
556,1,0,13,mild,Paroxetine,Full Remission,Improvement,Full Response
568,1,0,11,mild,Paroxetine,Full Remission,Improvement,Partial Response
588,1,0,12,mild,Paroxetine,Full Remission,Improvement,Full Response
607,1,0,12,mild,Paroxetine,Full Remission,Improvement,Partial Response
634,1,0,22,moderate,Paroxetine,Full Remission,Big Improvement,Full Response
...,...,...,...,...,...,...,...,...
1369,1,0,7,mild,Paroxetine,Full Remission,Improvement,Full Response
1374,2,0,23,moderate,Paroxetine,Partial Remission,Improvement,Partial Response
1389,1,0,13,mild,Paroxetine,Full Remission,Improvement,Full Response
1398,1,0,17,mild,Paroxetine,Full Remission,Improvement,Full Response


In [212]:
# Paroxetine Remission

mild_rp = len(r_op[r_op['Remission'] == 'No/Mild Remission'])
part_rp = len(r_op[r_op['Remission'] == 'Partial Remission'])
remi_rp = len(r_op[r_op['Remission'] == 'Remission'])
full_rp = len(r_op[r_op['Remission'] == 'Full Remission'])

# Paroxetine Difference
wor_rp = len(r_op[r_op['Difference'] == 'Worst'])
nc_rp = len(r_op[r_op['Difference'] == 'No Change'])
i_rp = len(r_op[r_op['Difference'] == 'Improvement'])
bi_rp = len(r_op[r_op['Difference'] == 'Big Improvement'])

# Paroxetine Response
nr_rp = len(r_op[r_op['Response'] == 'No Response'])
pr_rp = len(r_op[r_op['Response'] == 'Partial Response'])
fr_rp = len(r_op[r_op['Response'] == 'Full Response'])

rp = [mild_rp, part_rp, remi_rp, full_rp,
        wor_rp, nc_rp, i_rp, bi_rp,
        nr_rp, pr_rp, fr_rp]

rp

[1, 1, 24, 63, 1, 1, 78, 9, 2, 28, 59]

In [213]:
r_of = r_o[r_o['DRUG'] == 'Fluoxetine']

rof1 = go.Histogram(x = r_of['Remission'], name = 'Remission')
rof2 = go.Histogram(x = r_of['Difference'], name = 'Difference')
rof3 = go.Histogram(x = r_of['Response'], name = 'Response')

fig8 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig8.add_trace(rof1, row = 1, col = 1)
fig8.add_trace(rof2, row = 1, col = 2)
fig8.add_trace(rof3, row = 1, col = 3)

fig8.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig8.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig8.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig8.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 8. Retardation Only with Patients taking Fluoxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig8.show()

r_of

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
1434,1,0,19,moderate,Fluoxetine,Remission,Improvement,Partial Response
1435,1,0,17,mild,Fluoxetine,Full Remission,Improvement,Full Response
1436,1,0,13,mild,Fluoxetine,Full Remission,Improvement,Full Response
1445,1,0,21,moderate,Fluoxetine,Partial Remission,Improvement,Partial Response
1446,1,0,23,moderate,Fluoxetine,Partial Remission,No Change,No Response
1452,2,0,20,moderate,Fluoxetine,Remission,Improvement,Partial Response
1458,1,0,19,moderate,Fluoxetine,Remission,Improvement,Full Response
1461,1,0,21,moderate,Fluoxetine,Full Remission,Big Improvement,Full Response
1462,1,0,23,moderate,Fluoxetine,Remission,Improvement,Partial Response


In [214]:
# Fluoxetine Remission
mild_rf = len(r_of[r_of['Remission'] == 'No/Mild Remission'])
part_rf = len(r_of[r_of['Remission'] == 'Partial Remission'])
remi_rf = len(r_of[r_of['Remission'] == 'Remission'])
full_rf = len(r_of[r_of['Remission'] == 'Full Remission'])
                            
# Fluoxetine Difference
wor_rf = len(r_of[r_of['Difference'] == 'Worst'])
nc_rf = len(r_of[r_of['Difference'] == 'No Change'])
i_rf = len(r_of[r_of['Difference'] == 'Improvement'])
bi_rf = len(r_of[r_of['Difference'] == 'Big Improvement'])
                            
# Fluoxetine Response
nr_rf = len(r_of[r_of['Response'] == 'No Response'])
pr_rf = len(r_of[r_of['Response'] == 'Partial Response'])
fr_rf = len(r_of[r_of['Response'] == 'Full Response'])

rf = [mild_rf, part_rf, remi_rf, full_rf,
        wor_rf, nc_rf, i_rf, bi_rf,
        nr_rf, pr_rf, fr_rf]
rf

[0, 2, 4, 3, 0, 1, 7, 1, 1, 4, 4]

# Agitation and Retardation for each drug

In [215]:
r_a = df2.loc[((df['V1-HAMD08'] > 0) & (df['V1-HAMD09'] > 0))]
r_ad = r_a[r_a['DRUG'] == 'Duloxetine']

rad1 = go.Histogram(x = r_ad['Remission'], name = 'Remission')
rad2 = go.Histogram(x = r_ad['Difference'], name = 'Difference')
rad3 = go.Histogram(x = r_ad['Response'], name = 'Response')

fig9 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig9.add_trace(rad1, row = 1, col = 1)
fig9.add_trace(rad2, row = 1, col = 2)
fig9.add_trace(rad3, row = 1, col = 3)

fig9.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig9.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig9.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig9.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 9. Agitation and Retardation with Patients taking Duloxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig9.show()

r_a

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
2,1,1,22,moderate,Duloxetine,Full Remission,Improvement,Full Response
6,1,1,24,moderate,Duloxetine,Full Remission,Big Improvement,Full Response
8,2,1,16,mild,Duloxetine,Remission,Improvement,Partial Response
13,1,1,12,mild,Duloxetine,Full Remission,Improvement,Full Response
15,1,1,8,mild,Duloxetine,Full Remission,Improvement,Full Response
...,...,...,...,...,...,...,...,...
1460,1,2,17,mild,Fluoxetine,Remission,Improvement,Partial Response
1463,1,1,27,severe,Fluoxetine,Remission,Big Improvement,Full Response
1464,1,1,26,severe,Fluoxetine,Full Remission,Big Improvement,Full Response
1466,2,1,20,moderate,Fluoxetine,Remission,Improvement,Partial Response


In [216]:
# Duloxetine Remission
mild_rad = len(r_ad[r_ad['Remission'] == 'No/Mild Remission'])
part_rad= len(r_ad[r_ad['Remission'] == 'Partial Remission'])
remi_rad= len(r_ad[r_ad['Remission'] == 'Remission'])
full_rad= len(r_ad[r_ad['Remission'] == 'Full Remission'])

# Duloxetine Difference
wor_rad = len(r_ad[r_ad['Difference'] == 'Worst'])
nc_rad= len(r_ad[r_ad['Difference'] == 'No Change'])
i_rad= len(r_ad[r_ad['Difference'] == 'Improvement'])                            
bi_rad= len(r_ad[r_ad['Difference'] == 'Big Improvement'])

# Duloxetine Response
nr_rad = len(r_ad[r_ad['Response'] == 'No Response'])
pr_rad= len(r_ad[r_ad['Response'] == 'Partial Response'])
fr_rad= len(r_ad[r_ad['Response'] == 'Full Response'])

rad = [mild_rad, part_rad, remi_rad, full_rad,
        wor_rad, nc_rad, i_rad, bi_rad,
        nr_rad, pr_rad, fr_rad]

rad

[6, 42, 163, 156, 13, 8, 270, 76, 21, 142, 204]

In [217]:
r_av = r_a[r_a['DRUG'] == 'Venlafaxine']

rav1 = go.Histogram(x = r_av['Remission'], name = 'Remission')
rav2 = go.Histogram(x = r_av['Difference'], name = 'Difference')
rav3 = go.Histogram(x = r_av['Response'], name = 'Response')

fig10 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig10.add_trace(rav1, row = 1, col = 1)
fig10.add_trace(rav2, row = 1, col = 2)
fig10.add_trace(rav3, row = 1, col = 3)

fig10.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig10.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig10.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig10.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 10. Agitation and Retardation with Patients taking Venlafaxine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig10.show()

r_av

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
162,1,1,21,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
169,1,1,27,severe,Venlafaxine,Partial Remission,Improvement,Partial Response
172,1,2,24,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
174,1,1,25,severe,Venlafaxine,Remission,Improvement,Full Response
179,2,1,23,moderate,Venlafaxine,Remission,Improvement,Partial Response
...,...,...,...,...,...,...,...,...
403,2,2,28,severe,Venlafaxine,Remission,Improvement,Partial Response
405,1,2,23,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
406,1,1,21,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
416,1,1,20,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response


In [218]:
# Venlafaxine Remission

mild_rav = len(r_av[r_av['Remission'] == 'No/Mild Remission'])
part_rav= len(r_av[r_av['Remission'] == 'Partial Remission'])
remi_rav= len(r_av[r_av['Remission'] == 'Remission'])
full_rav= len(r_av[r_av['Remission'] == 'Full Remission'])

# Venlafaxine Difference
wor_rav = len(r_av[r_av['Difference'] == 'Worst'])
nc_rav= len(r_av[r_av['Difference'] == 'No Change'])
i_rav= len(r_av[r_av['Difference'] == 'Improvement'])
bi_rav= len(r_av[r_av['Difference'] == 'Big Improvement'])

# Venlafaxine Response
nr_rav = len(r_av[r_av['Response'] == 'No Response'])
pr_rav= len(r_av[r_av['Response'] == 'Partial Response'])
fr_rav= len(r_av[r_av['Response'] == 'Full Response'])

rav = [mild_rav, part_rav, remi_rav, full_rav,
        wor_rav, nc_rav, i_rav, bi_rav,
        nr_rav, pr_rav, fr_rav]

rav

[2, 6, 36, 28, 2, 1, 49, 20, 3, 29, 40]

In [219]:
r_ap = r_a[r_a['DRUG'] == 'Paroxetine']

rap1 = go.Histogram(x = r_ap['Remission'], name = 'Remission')
rap2 = go.Histogram(x = r_ap['Difference'], name = 'Difference')
rap3 = go.Histogram(x = r_ap['Response'], name = 'Response')

fig11 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig11.add_trace(rap1, row = 1, col = 1)
fig11.add_trace(rap2, row = 1, col = 2)
fig11.add_trace(rap3, row = 1, col = 3)

fig11.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig11.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig11.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig11.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 11. Agitation and Retardation with Patients taking Paroxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig11.show()

r_ap

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
557,1,1,17,mild,Paroxetine,Remission,Improvement,Partial Response
558,1,1,15,mild,Paroxetine,Full Remission,Improvement,Full Response
564,1,1,16,mild,Paroxetine,Remission,Improvement,Partial Response
570,1,1,11,mild,Paroxetine,Full Remission,Improvement,Full Response
585,1,1,20,moderate,Paroxetine,Full Remission,Big Improvement,Full Response
...,...,...,...,...,...,...,...,...
1396,1,1,21,moderate,Paroxetine,Full Remission,Big Improvement,Full Response
1400,1,1,22,moderate,Paroxetine,Full Remission,Big Improvement,Full Response
1404,2,1,20,moderate,Paroxetine,Full Remission,Big Improvement,Full Response
1414,1,1,19,moderate,Paroxetine,Full Remission,Improvement,Full Response


In [220]:
#Paroxetine Remission
mild_rap = len(r_ap[r_ap['Remission'] == 'No/Mild Remission'])
part_rap= len(r_ap[r_ap['Remission'] == 'Partial Remission'])
remi_rap= len(r_ap[r_ap['Remission'] == 'Remission'])
full_rap= len(r_ap[r_ap['Remission'] == 'Full Remission'])

# Paroxetine Difference
wor_rap = len(r_ap[r_ap['Difference'] == 'Worst'])
nc_rap= len(r_ap[r_ap['Difference'] == 'No Change'])
i_rap= len(r_ap[r_ap['Difference'] == 'Improvement'])
bi_rap= len(r_ap[r_ap['Difference'] == 'Big Improvement'])

# Paroxetine Response
nr_rap = len(r_ap[r_ap['Response'] == 'No Response'])
pr_rap= len(r_ap[r_ap['Response'] == 'Partial Response'])
fr_rap= len(r_ap[r_ap['Response'] == 'Full Response'])

rap = [mild_rap, part_rap, remi_rap, full_rap,
        wor_rap, nc_rap, i_rap, bi_rap,
        nr_rap, pr_rap, fr_rap]

rap

[4, 12, 67, 62, 9, 5, 114, 17, 14, 60, 71]

In [221]:
r_af = r_a[r_a['DRUG'] == 'Fluoxetine']

raf1 = go.Histogram(x = r_af['Remission'], name = 'Remission')
raf2 = go.Histogram(x = r_af['Difference'], name = 'Difference')
raf3 = go.Histogram(x = r_af['Response'], name = 'Response')

fig12 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig12.add_trace(raf1, row = 1, col = 1)
fig12.add_trace(raf2, row = 1, col = 2)
fig12.add_trace(raf3, row = 1, col = 3)

fig12.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig12.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig12.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig12.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 12. Agitation and Retardation with Patients taking Fluoxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig12.show()

r_af

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
1424,1,1,17,mild,Fluoxetine,Remission,Improvement,Full Response
1425,1,1,20,moderate,Fluoxetine,Remission,Improvement,Partial Response
1438,1,1,21,moderate,Fluoxetine,Remission,Improvement,Partial Response
1443,1,1,16,mild,Fluoxetine,Remission,Improvement,Partial Response
1444,1,1,23,moderate,Fluoxetine,No/Mild Remission,Worst,No Response
1447,1,1,17,mild,Fluoxetine,Remission,Improvement,Partial Response
1449,1,1,21,moderate,Fluoxetine,Full Remission,Big Improvement,Full Response
1450,1,1,17,mild,Fluoxetine,Full Remission,Improvement,Full Response
1453,1,2,24,moderate,Fluoxetine,Remission,Improvement,Partial Response
1456,2,1,24,moderate,Fluoxetine,Full Remission,Big Improvement,Full Response


In [222]:
# Fluoxetine Remission

mild_raf = len(r_af[r_af['Remission'] == 'No/Mild Remission'])
part_raf= len(r_af[r_af['Remission'] == 'Partial Remission'])
remi_raf= len(r_af[r_af['Remission'] == 'Remission'])
full_raf= len(r_af[r_af['Remission'] == 'Full Remission'])
                            
# Fluoxetine Difference
wor_raf = len(r_af[r_af['Difference'] == 'Worst'])
nc_raf= len(r_af[r_af['Difference'] == 'No Change'])
i_raf= len(r_af[r_af['Difference'] == 'Improvement'])
bi_raf= len(r_af[r_af['Difference'] == 'Big Improvement'])
                            
#Fluoxetine Response
nr_raf = len(r_af[r_af['Response'] == 'No Response'])
pr_raf= len(r_af[r_af['Response'] == 'Partial Response'])
fr_raf= len(r_af[r_af['Response'] == 'Full Response'])

raf = [mild_raf, part_raf, remi_raf, full_raf,
        wor_raf, nc_raf, i_raf, bi_raf,
        nr_raf, pr_raf, fr_raf]

raf

[1, 1, 9, 5, 2, 0, 9, 5, 2, 7, 7]

# Neither Agitation nor Retardation for each drug

In [223]:
na_r = df2.loc[((df['V1-HAMD08'] == 0) & (df['V1-HAMD09'] == 0))]
na_rd = na_r[na_r['DRUG'] == 'Duloxetine']

nard1 = go.Histogram(x = na_rd['Remission'], name = 'Remission')
nard2 = go.Histogram(x = na_rd['Difference'], name = 'Difference')
nard3 = go.Histogram(x = na_rd['Response'], name = 'Response')

fig13 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig13.add_trace(nard1, row = 1, col = 1)
fig13.add_trace(nard2, row = 1, col = 2)
fig13.add_trace(nard3, row = 1, col = 3)

fig13.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig13.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig13.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig13.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 13. Neither Agitation nor Retardation with Patients taking Duloxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig13.show()

na_rd

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
0,0,0,7,mild,Duloxetine,Full Remission,Improvement,Full Response
3,0,0,6,mild,Duloxetine,Remission,Worst,No Response
4,0,0,8,mild,Duloxetine,Remission,Worst,No Response
5,0,0,6,mild,Duloxetine,Full Remission,Improvement,Full Response
7,0,0,13,mild,Duloxetine,Remission,Improvement,Partial Response
...,...,...,...,...,...,...,...,...
1234,0,0,13,mild,Duloxetine,Full Remission,Improvement,Full Response
1255,0,0,12,mild,Duloxetine,Remission,Improvement,Partial Response
1371,0,0,13,mild,Duloxetine,Remission,Improvement,Partial Response
1378,0,0,8,mild,Duloxetine,Full Remission,Improvement,Partial Response


In [224]:
# Duloxetine Remission
mild_nard = len(na_rd[na_rd['Remission'] == 'No/Mild Remission'])
part_nard = len(na_rd[na_rd['Remission'] == 'Partial Remission'])
remi_nard = len(na_rd[na_rd['Remission'] == 'Remission'])
full_nard = len(na_rd[na_rd['Remission'] == 'Full Remission'])

# Duloxetine Difference
wor_nard = len(na_rd[na_rd['Difference'] == 'Worst'])
nc_nard = len(na_rd[na_rd['Difference'] == 'No Change'])
i_nard = len(na_rd[na_rd['Difference'] == 'Improvement'])
bi_nard = len(na_rd[na_rd['Difference'] == 'Big Improvement'])

# Duloxetine Response
nr_nard = len(na_rd[na_rd['Response'] == 'No Response'])
pr_nard = len(na_rd[na_rd['Response'] == 'Partial Response'])
fr_nard = len(na_rd[na_rd['Response'] == 'Full Response'])

nard = [mild_nard, part_nard, remi_nard, full_nard,
        wor_nard, nc_nard, i_nard, bi_nard,
        nr_nard, pr_nard, fr_nard]

nard

[2, 6, 40, 73, 19, 8, 89, 5, 27, 35, 59]

In [225]:
na_rv = na_r[na_r['DRUG'] == 'Venlafaxine']

narv1 = go.Histogram(x = na_rv['Remission'], name = 'Remission')
narv2 = go.Histogram(x = na_rv['Difference'], name = 'Difference')
narv3 = go.Histogram(x = na_rv['Response'], name = 'Response')

fig14 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig14.add_trace(narv1, row = 1, col = 1)
fig14.add_trace(narv2, row = 1, col = 2)
fig14.add_trace(narv3, row = 1, col = 3)

fig14.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig14.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig14.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig14.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 14. Neither Agitation nor Retardation with Patients taking Venlafaxine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig14.show()

na_rv

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
180,0,0,27,severe,Venlafaxine,Partial Remission,Improvement,Partial Response
189,0,0,27,severe,Venlafaxine,Remission,Big Improvement,Full Response
191,0,0,21,moderate,Venlafaxine,Remission,Improvement,Partial Response
194,0,0,17,mild,Venlafaxine,Full Remission,Big Improvement,Full Response
197,0,0,20,moderate,Venlafaxine,Full Remission,Big Improvement,Full Response
205,0,0,20,moderate,Venlafaxine,Remission,Improvement,Partial Response
292,0,0,19,moderate,Venlafaxine,Remission,Improvement,Full Response
348,0,0,20,moderate,Venlafaxine,Full Remission,Improvement,Full Response
414,0,0,21,moderate,Venlafaxine,Full Remission,Improvement,Full Response
415,0,0,24,moderate,Venlafaxine,Remission,Big Improvement,Full Response


In [226]:
# Venlafaxine Remission
mild_narv = len(na_rv[na_rv['Remission'] == 'No/Mild Remission'])
part_narv = len(na_rv[na_rv['Remission'] == 'Partial Remission'])
remi_narv = len(na_rv[na_rv['Remission'] == 'Remission'])
full_narv = len(na_rv[na_rv['Remission'] == 'Full Remission'])

# Venlafaxine Difference
wor_narv = len(na_rv[na_rv['Difference'] == 'Worst'])
nc_narv = len(na_rv[na_rv['Difference'] == 'No Change'])
i_narv = len(na_rv[na_rv['Difference'] == 'Improvement'])
bi_narv = len(na_rv[na_rv['Difference'] == 'Big Improvement'])

# Venlafaxine Response
nr_narv = len(na_rv[na_rv['Response'] == 'No Response'])
pr_narv = len(na_rv[na_rv['Response'] == 'Partial Response'])
fr_narv = len(na_rv[na_rv['Response'] == 'Full Response'])

narv = [mild_narv, part_narv, remi_narv, full_narv,
        wor_narv, nc_narv, i_narv, bi_narv,
        nr_narv, pr_narv, fr_narv]

narv

[0, 1, 5, 4, 0, 0, 6, 4, 0, 3, 7]

In [227]:
na_rp = na_r[na_r['DRUG'] == 'Paroxetine']

narp1 = go.Histogram(x = na_rp['Remission'], name = 'Remission')
narp2 = go.Histogram(x = na_rp['Difference'], name = 'Difference')
narp3 = go.Histogram(x = na_rp['Response'], name = 'Response')

fig15 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig15.add_trace(narp1, row = 1, col = 1)
fig15.add_trace(narp2, row = 1, col = 2)
fig15.add_trace(narp3, row = 1, col = 3)

fig15.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig15.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig15.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig15.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 15. Neither Agitation nor Retardation with Patients taking Paroxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig15.show()

na_rp

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
555,0,0,6,mild,Paroxetine,Full Remission,Improvement,Full Response
565,0,0,9,mild,Paroxetine,Remission,Worst,No Response
575,0,0,17,mild,Paroxetine,Remission,Improvement,Partial Response
576,0,0,17,mild,Paroxetine,Full Remission,Improvement,Full Response
579,0,0,10,mild,Paroxetine,Full Remission,Improvement,Full Response
600,0,0,7,mild,Paroxetine,Full Remission,Improvement,Partial Response
602,0,0,10,mild,Paroxetine,Full Remission,Improvement,Full Response
612,0,0,7,mild,Paroxetine,Remission,Worst,No Response
616,0,0,7,mild,Paroxetine,Full Remission,Improvement,Partial Response
626,0,0,11,mild,Paroxetine,Remission,Improvement,Partial Response


In [228]:
# Paroxetine Remission
mild_narp = len(na_rp[na_rp['Remission'] == 'No/Mild Remission'])
part_narp = len(na_rp[na_rp['Remission'] == 'Partial Remission'])
remi_narp = len(na_rp[na_rp['Remission'] == 'Remission'])
full_narp = len(na_rp[na_rp['Remission'] == 'Full Remission'])

# Paroxetine Difference
wor_narp = len(na_rp[na_rp['Difference'] == 'Worst'])
nc_narp = len(na_rp[na_rp['Difference'] == 'No Change'])
i_narp = len(na_rp[na_rp['Difference'] == 'Improvement'])
bi_narp = len(na_rp[na_rp['Difference'] == 'Big Improvement'])

# Paroxetine Response
nr_narp = len(na_rp[na_rp['Response'] == 'No Response'])
pr_narp = len(na_rp[na_rp['Response'] == 'Partial Response'])
fr_narp = len(na_rp[na_rp['Response'] == 'Full Response'])

narp = [mild_narp, part_narp, remi_narp, full_narp,
        wor_narp, nc_narp, i_narp, bi_narp,
        nr_narp, pr_narp, fr_narp]

narp

[1, 2, 16, 38, 8, 1, 45, 3, 9, 21, 27]

In [229]:
na_rf = na_r[na_r['DRUG'] == 'Fluoxetine']

narf1 = go.Histogram(x = na_rf['Remission'], name = 'Remission')
narf2 = go.Histogram(x = na_rf['Difference'], name = 'Difference')
narf3 = go.Histogram(x = na_rf['Response'], name = 'Response')

fig16 = make_subplots(rows = 1, cols = 3, shared_yaxes = True, horizontal_spacing = 0.02)
fig16.add_trace(narf1, row = 1, col = 1)
fig16.add_trace(narf2, row = 1, col = 2)
fig16.add_trace(narf3, row = 1, col = 3)

fig16.update_xaxes(categoryorder = 'array', categoryarray = remission_labels, row = 1, col = 1)
fig16.update_xaxes(categoryorder = 'array', categoryarray = difference_labels, row = 1, col = 2)
fig16.update_xaxes(categoryorder = 'array', categoryarray = response_labels, row = 1, col = 3)

fig16.update_layout(
    font = {
        "size": 15
    },
    
    title = {
        "text": 'Figure 16. Neither Agitation nor Retardation with Patients taking Fluoxetine',
        "x": 0.5,
        "xanchor" : "center",
        "font" : {
            "size": 20
        }
    },
    #xaxis =dict(tickmode = 'linear',
    #tick0 = 0,
    #dtick = 1),
    xaxis1 = {
        "tickangle": -45,
    },
    xaxis2 = {
        "title" : 'Drug Efficiency Labels',
        "tickangle": -45,
    },
    xaxis3 = {
        "tickangle": -45,
    },
#     xaxis_title = 'Drug Efficiency Labels',
    yaxis_title = 'Count',
    showlegend = True,
    width = 1600,
    height = 900)

fig16.show()

na_rf
# len(na_rf[na_rf['Remission'] == 'Remission'])

,V1-HAMD08,V1-HAMD09,V1-all,level,DRUG,Remission,Difference,Response
1427,0,0,11,mild,Fluoxetine,Remission,No Change,No Response
1429,0,0,13,mild,Fluoxetine,Remission,Worst,No Response
1437,0,0,16,mild,Fluoxetine,Full Remission,Big Improvement,Full Response
1440,0,0,19,moderate,Fluoxetine,Full Remission,Big Improvement,Full Response
1441,0,0,11,mild,Fluoxetine,Full Remission,Improvement,Full Response
1451,0,0,15,mild,Fluoxetine,Full Remission,Improvement,Full Response


In [230]:
# Fluoxetine Remission
mild_narf = len(na_rf[na_rf['Remission'] == 'No/Mild Remission'])
part_narf = len(na_rf[na_rf['Remission'] == 'Partial Remission'])
remi_narf = len(na_rf[na_rf['Remission'] == 'Remission'])
full_narf = len(na_rf[na_rf['Remission'] == 'Full Remission'])

#Fluoxetine Difference
wor_narf = len(na_rf[na_rf['Difference'] == 'Worst'])
nc_narf = len(na_rf[na_rf['Difference'] == 'No Change'])
i_narf = len(na_rf[na_rf['Difference'] == 'Improvement'])
bi_narf = len(na_rf[na_rf['Difference'] == 'Big Improvement'])

# Fluoxetine Response
nr_narf = len(na_rf[na_rf['Response'] == 'No Response'])
pr_narf = len(na_rf[na_rf['Response'] == 'Partial Response'])
fr_narf = len(na_rf[na_rf['Response'] == 'Full Response'])

narf = [mild_narf, part_narf, remi_narf, full_narf,
        wor_narf, nc_narf, i_narf, bi_narf,
        nr_narf, pr_narf, fr_narf]

narf

[0, 0, 2, 4, 1, 1, 2, 2, 2, 0, 4]

# The Statistics Table

In [231]:
Labels = ['Agitation Only, Duloxetine', 'Agitation Only, Venlaxafine','Agitation Only, Paroxetine', 'Agitation Only, Fluoxine',
          'Retardation Only, Duloxetine', 'Retardation Only, Venlaxafine','Retardation Only, Paroxetine', 'Retardation Only, Fluoxine',
          'Agitation and Retardation, Duloxetine', 'Agitation and Retardation, Venlaxafine','Agitation and Retardation, Paroxetine', 'Agitation and Retardation, Fluoxine',
          'Neither Agitation nor Retardation, Duloxetine', 'Neither Agitation nor Retardation, Venlaxafine','Neither Agitation nor Retardation, Paroxetine', 'Neither Agitation nor Retardation, Fluoxine']

header = [('Remission', 'No/Mild Remission'), ('Remission', 'Partial Remission'), ('Remission', 'Remission'), ('Remission', 'Full Remission'),
         ('Difference', 'Worst'), ('Difference', 'No Change'), ('Difference', 'Improvement'), ('Difference', 'Big Improvement'),
         ('Response', 'No Response'), ('Response', 'Partial Response'), ('Response', 'Full Response')]

data = pd.DataFrame([ad, av, ap, af,
                     rd, rv, rp, rf, 
                     rad, rav, rap, raf,
                     nard, narv, narp, narf],
                   index = Labels)

data.columns = pd.MultiIndex.from_tuples(header)

data.to_csv("Mission 2.csv")
# {'Remission': ['No/Mild Remission': [len(na_rf[na_rf['Remission'] == 'No/Mild Remission'])],
#            'Partial Remission': [len(na_rf[na_rf['Remission'] == 'Partial Remission'])],
#            'Remission': [len(na_rf[na_rf['Remission'] == 'Remission'])],
#            'Full Remission': [len(na_rf[na_rf['Remission'] == 'Full Remission'])]]}
#['Difference', 'Difference', 'Difference', 'Difference'], ['Worst','No Change', 'Improvement', 'Big Improvement']

data

Remission  \
                                                   No/Mild Remission   
Agitation Only, Duloxetine                                         5   
Agitation Only, Venlaxafine                                        0   
Agitation Only, Paroxetine                                         1   
Agitation Only, Fluoxine                                           0   
Retardation Only, Duloxetine                                       4   
Retardation Only, Venlaxafine                                      0   
Retardation Only, Paroxetine                                       1   
Retardation Only, Fluoxine                                         0   
Agitation and Retardation, Duloxetine                              6   
Agitation and Retardation, Venlaxafine                             2   
Agitation and Retardation, Paroxetine                              4   
Agitation and Retardation, Fluoxine                                1   
Neither Agitation nor Retardation Only, Duloxetine                 2   
Neither Agitation nor Retardation, Venlaxafine                     0   
Neither Agitation nor Retardation, Paroxetine                      1   
Neither Agitation nor Retardation, Fluoxine                        0   

                                                                      \
                                                   Partial Remission   
Agitation Only, Duloxetine                                        11   
Agitation Only, Venlaxafine                                        4   
Agitation Only, Paroxetine                                         2   
Agitation Only, Fluoxine                                           3   
Retardation Only, Duloxetine                                      14   
Retardation Only, Venlaxafine                                      4   
Retardation Only, Paroxetine                                       1   
Retardation Only, Fluoxine                                         2   
Agitation and Retardation, Duloxetine                             42   
Agitation and Retardation, Venlaxafine                             6   
Agitation and Retardation, Paroxetine                             12   
Agitation and Retardation, Fluoxine                                1   
Neither Agitation nor Retardation Only, Duloxetine                 6   
Neither Agitation nor Retardation, Venlaxafine                     1   
Neither Agitation nor Retardation, Paroxetine                      2   
Neither Agitation nor Retardation, Fluoxine                        0   

                                                                             \
                                                   Remission Full Remission   
Agitation Only, Duloxetine                                86             89   
Agitation Only, Venlaxafine                               18              1   
Agitation Only, Paroxetine                                25             33   
Agitation Only, Fluoxine                                   7              4   
Retardation Only, Duloxetine                             113            111   
Retardation Only, Venlaxafine                             15             15   
Retardation Only, Paroxetine                              24             63   
Retardation Only, Fluoxine                                 4              3   
Agitation and Retardation, Duloxetine                    163            156   
Agitation and Retardation, Venlaxafine                    36             28   
Agitation and Retardation, Paroxetine                     67             62   
Agitation and Retardation, Fluoxine                        9              5   
Neither Agitation nor Retardation Only, Duloxetine        40             73   
Neither Agitation nor Retardation, Venlaxafine             5              4   
Neither Agitation nor Retardation, Paroxetine             16             38   
Neither Agitation nor Retardation, Fluoxine                2              4   

                                          

# Remission for each drugs

In [232]:
# No/Mild Remission

mild_ad = len(a_od[a_od['Remission'] == 'No/Mild Remission'])
mild_av = len(a_ov[a_ov['Remission'] == 'No/Mild Remission'])
mild_ap = len(a_op[a_op['Remission'] == 'No/Mild Remission'])
mild_af = len(a_of[a_of['Remission'] == 'No/Mild Remission'])
                             
mild_rd = len(r_od[r_od['Remission'] == 'No/Mild Remission'])
mild_rv = len(r_ov[r_ov['Remission'] == 'No/Mild Remission'])
mild_rp = len(r_op[r_op['Remission'] == 'No/Mild Remission'])
mild_rf = len(r_of[r_of['Remission'] == 'No/Mild Remission'])
                            
mild_rad = len(r_ad[r_ad['Remission'] == 'No/Mild Remission'])
mild_rav = len(r_av[r_av['Remission'] == 'No/Mild Remission'])
mild_rap = len(r_ap[r_ap['Remission'] == 'No/Mild Remission'])
mild_raf = len(r_af[r_af['Remission'] == 'No/Mild Remission'])
                            
mild_nard = len(na_rd[na_rd['Remission'] == 'No/Mild Remission'])
mild_narv = len(na_rv[na_rv['Remission'] == 'No/Mild Remission'])
mild_narp = len(na_rp[na_rp['Remission'] == 'No/Mild Remission'])
mild_narf = len(na_rf[na_rf['Remission'] == 'No/Mild Remission'])

mild = [mild_ad,mild_av, mild_ap, mild_af,
        mild_rd, mild_rv, mild_rp, mild_rf,
        mild_rad, mild_rav, mild_rap, mild_raf,
        mild_nard, mild_narv, mild_narp, mild_narf]

# Partial Remission

part_ad = len(a_od[a_od['Remission'] == 'Partial Remission'])
part_av = len(a_ov[a_ov['Remission'] == 'Partial Remission'])
part_ap = len(a_op[a_op['Remission'] == 'Partial Remission'])
part_af = len(a_of[a_of['Remission'] == 'Partial Remission'])
                             
part_rd = len(r_od[r_od['Remission'] == 'Partial Remission'])
part_rv = len(r_ov[r_ov['Remission'] == 'Partial Remission'])
part_rp = len(r_op[r_op['Remission'] == 'Partial Remission'])
part_rf = len(r_of[r_of['Remission'] == 'Partial Remission'])
                            
part_rad= len(r_ad[r_ad['Remission'] == 'Partial Remission'])
part_rav= len(r_av[r_av['Remission'] == 'Partial Remission'])
part_rap= len(r_ap[r_ap['Remission'] == 'Partial Remission'])
part_raf= len(r_af[r_af['Remission'] == 'Partial Remission'])
                            
part_nard = len(na_rd[na_rd['Remission'] == 'Partial Remission'])
part_narv = len(na_rv[na_rv['Remission'] == 'Partial Remission'])
part_narp = len(na_rp[na_rp['Remission'] == 'Partial Remission'])
part_narf = len(na_rf[na_rf['Remission'] == 'Partial Remission'])

part = [part_ad,part_av, part_ap, part_af,
        part_rd, part_rv, part_rp, part_rf,
        part_rad, part_rav, part_rap, part_raf,
        part_nard, part_narv, part_narp, part_narf]

#Remission

remi_ad = len(a_od[a_od['Remission'] == 'Remission'])
remi_av = len(a_ov[a_ov['Remission'] == 'Remission'])
remi_ap = len(a_op[a_op['Remission'] == 'Remission'])
remi_af = len(a_of[a_of['Remission'] == 'Remission'])
                             
remi_rd = len(r_od[r_od['Remission'] == 'Remission'])
remi_rv = len(r_ov[r_ov['Remission'] == 'Remission'])
remi_rp = len(r_op[r_op['Remission'] == 'Remission'])
remi_rf = len(r_of[r_of['Remission'] == 'Remission'])
                            
remi_rad= len(r_ad[r_ad['Remission'] == 'Remission'])
remi_rav= len(r_av[r_av['Remission'] == 'Remission'])
remi_rap= len(r_ap[r_ap['Remission'] == 'Remission'])
remi_raf= len(r_af[r_af['Remission'] == 'Remission'])
                            
remi_nard = len(na_rd[na_rd['Remission'] == 'Remission'])
remi_narv = len(na_rv[na_rv['Remission'] == 'Remission'])
remi_narp = len(na_rp[na_rp['Remission'] == 'Remission'])
remi_narf = len(na_rf[na_rf['Remission'] == 'Remission'])

remi = [remi_ad,remi_av, remi_ap, remi_af,
        remi_rd, remi_rv, remi_rp, remi_rf,
        remi_rad, remi_rav, remi_rap, remi_raf,
        remi_nard, remi_narv, remi_narp, remi_narf]

#Full Remission

full_ad = len(a_od[a_od['Remission'] == 'Full Remission'])
full_av = len(a_ov[a_ov['Remission'] == 'Full Remission'])
full_ap = len(a_op[a_op['Remission'] == 'Full Remission'])
full_af = len(a_of[a_of['Remission'] == 'Full Remission'])
                             
full_rd = len(r_od[r_od['Remission'] == 'Full Remission'])
full_rv = len(r_ov[r_ov['Remission'] == 'Full Remission'])
full_rp = len(r_op[r_op['Remission'] == 'Full Remission'])
full_rf = len(r_of[r_of['Remission'] == 'Full Remission'])
                            
full_rad= len(r_ad[r_ad['Remission'] == 'Full Remission'])
full_rav= len(r_av[r_av['Remission'] == 'Full Remission'])
full_rap= len(r_ap[r_ap['Remission'] == 'Full Remission'])
full_raf= len(r_af[r_af['Remission'] == 'Full Remission'])
                            
full_nard = len(na_rd[na_rd['Remission'] == 'Full Remission'])
full_narv = len(na_rv[na_rv['Remission'] == 'Full Remission'])
full_narp = len(na_rp[na_rp['Remission'] == 'Full Remission'])
full_narf = len(na_rf[na_rf['Remission'] == 'Full Remission'])

full = [full_ad,full_av, full_ap, full_af,
        full_rd, full_rv, full_rp, full_rf,
        full_rad, full_rav, full_rap, full_raf,
        full_nard, full_narv, full_narp, full_narf]



# Worse Difference

wor_ad = len(a_od[a_od['Difference'] == 'Worst'])
wor_av = len(a_ov[a_ov['Difference'] == 'Worst'])
wor_ap = len(a_op[a_op['Difference'] == 'Worst'])
wor_af = len(a_of[a_of['Difference'] == 'Worst'])
                             
wor_rd = len(r_od[r_od['Difference'] == 'Worst'])
wor_rv = len(r_ov[r_ov['Difference'] == 'Worst'])
wor_rp = len(r_op[r_op['Difference'] == 'Worst'])
wor_rf = len(r_of[r_of['Difference'] == 'Worst'])
                            
wor_rad = len(r_ad[r_ad['Difference'] == 'Worst'])
wor_rav = len(r_av[r_av['Difference'] == 'Worst'])
wor_rap = len(r_ap[r_ap['Difference'] == 'Worst'])
wor_raf = len(r_af[r_af['Difference'] == 'Worst'])
                            
wor_nard = len(na_rd[na_rd['Difference'] == 'Worst'])
wor_narv = len(na_rv[na_rv['Difference'] == 'Worst'])
wor_narp = len(na_rp[na_rp['Difference'] == 'Worst'])
wor_narf = len(na_rf[na_rf['Difference'] == 'Worst'])

# mild = [mild_ad,mild_av, mild_ap, mild_af,
#         mild_rd, mild_rv, mild_rp, mild_rf,
#         mild_rad, mild_rav, mild_rap, mild_raf,
#         mild_nard, mild_narv, mild_narp, mild_narf]

# No Change

nc_ad = len(a_od[a_od['Difference'] == 'No Change'])
nc_av = len(a_ov[a_ov['Difference'] == 'No Change'])
nc_ap = len(a_op[a_op['Difference'] == 'No Change'])
nc_af = len(a_of[a_of['Difference'] == 'No Change'])
                             
nc_rd = len(r_od[r_od['Difference'] == 'No Change'])
nc_rv = len(r_ov[r_ov['Difference'] == 'No Change'])
nc_rp = len(r_op[r_op['Difference'] == 'No Change'])
nc_rf = len(r_of[r_of['Difference'] == 'No Change'])
                            
nc_rad= len(r_ad[r_ad['Difference'] == 'No Change'])
nc_rav= len(r_av[r_av['Difference'] == 'No Change'])
nc_rap= len(r_ap[r_ap['Difference'] == 'No Change'])
nc_raf= len(r_af[r_af['Difference'] == 'No Change'])
                            
nc_nard = len(na_rd[na_rd['Difference'] == 'No Change'])
nc_narv = len(na_rv[na_rv['Difference'] == 'No Change'])
nc_narp = len(na_rp[na_rp['Difference'] == 'No Change'])
nc_narf = len(na_rf[na_rf['Difference'] == 'No Change'])

# part = [part_ad,part_av, part_ap, part_af,
#         part_rd, part_rv, part_rp, part_rf,
#         part_rad, part_rav, part_rap, part_raf,
#         part_nard, part_narv, part_narp, part_narf]

#Improvement

i_ad = len(a_od[a_od['Difference'] == 'Improvement'])
i_av = len(a_ov[a_ov['Difference'] == 'Improvement'])
i_ap = len(a_op[a_op['Difference'] == 'Improvement'])
i_af = len(a_of[a_of['Difference'] == 'Improvement'])
                             
i_rd = len(r_od[r_od['Difference'] == 'Improvement'])
i_rv = len(r_ov[r_ov['Difference'] == 'Improvement'])
i_rp = len(r_op[r_op['Difference'] == 'Improvement'])
i_rf = len(r_of[r_of['Difference'] == 'Improvement'])
                            
i_rad= len(r_ad[r_ad['Difference'] == 'Improvement'])
i_rav= len(r_av[r_av['Difference'] == 'Improvement'])
i_rap= len(r_ap[r_ap['Difference'] == 'Improvement'])
i_raf= len(r_af[r_af['Difference'] == 'Improvement'])
                            
i_nard = len(na_rd[na_rd['Difference'] == 'Improvement'])
i_narv = len(na_rv[na_rv['Difference'] == 'Improvement'])
i_narp = len(na_rp[na_rp['Difference'] == 'Improvement'])
i_narf = len(na_rf[na_rf['Difference'] == 'Improvement'])

# remi = [remi_ad,remi_av, remi_ap, remi_af,
#         remi_rd, remi_rv, remi_rp, remi_rf,
#         remi_rad, remi_rav, remi_rap, remi_raf,
#         remi_nard, remi_narv, remi_narp, remi_narf]

#Big Improvement

bi_ad = len(a_od[a_od['Difference'] == 'Big Improvement'])
bi_av = len(a_ov[a_ov['Difference'] == 'Big Improvement'])
bi_ap = len(a_op[a_op['Difference'] == 'Big Improvement'])
bi_af = len(a_of[a_of['Difference'] == 'Big Improvement'])
                             
bi_rd = len(r_od[r_od['Difference'] == 'Big Improvement'])
bi_rv = len(r_ov[r_ov['Difference'] == 'Big Improvement'])
bi_rp = len(r_op[r_op['Difference'] == 'Big Improvement'])
bi_rf = len(r_of[r_of['Difference'] == 'Big Improvement'])
                            
bi_rad= len(r_ad[r_ad['Difference'] == 'Big Improvement'])
bi_rav= len(r_av[r_av['Difference'] == 'Big Improvement'])
bi_rap= len(r_ap[r_ap['Difference'] == 'Big Improvement'])
bi_raf= len(r_af[r_af['Difference'] == 'Big Improvement'])
                            
bi_nard = len(na_rd[na_rd['Difference'] == 'Big Improvement'])
bi_narv = len(na_rv[na_rv['Difference'] == 'Big Improvement'])
bi_narp = len(na_rp[na_rp['Difference'] == 'Big Improvement'])
bi_narf = len(na_rf[na_rf['Difference'] == 'Big Improvement'])

# full = [full_ad,full_av, full_ap, full_af,
#         full_rd, full_rv, full_rp, full_rf,
#         full_rad, full_rav, full_rap, full_raf,
#         full_nard, full_narv, full_narp, full_narf]

# No Response

nr_ad = len(a_od[a_od['Response'] == 'No Response'])
nr_av = len(a_ov[a_ov['Response'] == 'No Response'])
nr_ap = len(a_op[a_op['Response'] == 'No Response'])
nr_af = len(a_of[a_of['Response'] == 'No Response'])
                             
nr_rd = len(r_od[r_od['Response'] == 'No Response'])
nr_rv = len(r_ov[r_ov['Response'] == 'No Response'])
nr_rp = len(r_op[r_op['Response'] == 'No Response'])
nr_rf = len(r_of[r_of['Response'] == 'No Response'])
                            
nr_rad = len(r_ad[r_ad['Response'] == 'No Response'])
nr_rav = len(r_av[r_av['Response'] == 'No Response'])
nr_rap = len(r_ap[r_ap['Response'] == 'No Response'])
nr_raf = len(r_af[r_af['Response'] == 'No Response'])
                            
nr_nard = len(na_rd[na_rd['Response'] == 'No Response'])
nr_narv = len(na_rv[na_rv['Response'] == 'No Response'])
nr_narp = len(na_rp[na_rp['Response'] == 'No Response'])
nr_narf = len(na_rf[na_rf['Response'] == 'No Response'])

# mild = [mild_ad,mild_av, mild_ap, mild_af,
#         mild_rd, mild_rv, mild_rp, mild_rf,
#         mild_rad, mild_rav, mild_rap, mild_raf,
#         mild_nard, mild_narv, mild_narp, mild_narf]

# Partial Response

pr_ad = len(a_od[a_od['Response'] == 'Partial Response'])
pr_av = len(a_ov[a_ov['Response'] == 'Partial Response'])
pr_ap = len(a_op[a_op['Response'] == 'Partial Response'])
pr_af = len(a_of[a_of['Response'] == 'Partial Response'])
                             
pr_rd = len(r_od[r_od['Response'] == 'Partial Response'])
pr_rv = len(r_ov[r_ov['Response'] == 'Partial Response'])
pr_rp = len(r_op[r_op['Response'] == 'Partial Response'])
pr_rf = len(r_of[r_of['Response'] == 'Partial Response'])
                            
pr_rad= len(r_ad[r_ad['Response'] == 'Partial Response'])
pr_rav= len(r_av[r_av['Response'] == 'Partial Response'])
pr_rap= len(r_ap[r_ap['Response'] == 'Partial Response'])
pr_raf= len(r_af[r_af['Response'] == 'Partial Response'])
                            
pr_nard = len(na_rd[na_rd['Response'] == 'Partial Response'])
pr_narv = len(na_rv[na_rv['Response'] == 'Partial Response'])
pr_narp = len(na_rp[na_rp['Response'] == 'Partial Response'])
pr_narf = len(na_rf[na_rf['Response'] == 'Partial Response'])

# part = [part_ad,part_av, part_ap, part_af,
#         part_rd, part_rv, part_rp, part_rf,
#         part_rad, part_rav, part_rap, part_raf,
#         part_nard, part_narv, part_narp, part_narf]

# Full Response

fr_ad = len(a_od[a_od['Response'] == 'Full Response'])
fr_av = len(a_ov[a_ov['Response'] == 'Full Response'])
fr_ap = len(a_op[a_op['Response'] == 'Full Response'])
fr_af = len(a_of[a_of['Response'] == 'Full Response'])
                             
fr_rd = len(r_od[r_od['Response'] == 'Full Response'])
fr_rv = len(r_ov[r_ov['Response'] == 'Full Response'])
fr_rp = len(r_op[r_op['Response'] == 'Full Response'])
fr_rf = len(r_of[r_of['Response'] == 'Full Response'])
                            
fr_rad= len(r_ad[r_ad['Response'] == 'Full Response'])
fr_rav= len(r_av[r_av['Response'] == 'Full Response'])
fr_rap= len(r_ap[r_ap['Response'] == 'Full Response'])
fr_raf= len(r_af[r_af['Response'] == 'Full Response'])
                            
fr_nard = len(na_rd[na_rd['Response'] == 'Full Response'])
fr_narv = len(na_rv[na_rv['Response'] == 'Full Response'])
fr_narp = len(na_rp[na_rp['Response'] == 'Full Response'])
fr_narf = len(na_rf[na_rf['Response'] == 'Full Response'])

# remi = [remi_ad,remi_av, remi_ap, remi_af,
#         remi_rd, remi_rv, remi_rp, remi_rf,
#         remi_rad, remi_rav, remi_rap, remi_raf,
#         remi_nard, remi_narv, remi_narp, remi_narf]

# full = [full_ad,full_av, full_ap, full_af,
#         full_rd, full_rv, full_rp, full_rf,
#         full_rad, full_rav, full_rap, full_raf,
#         full_nard, full_narv, full_narp, full_narf]